In [ ]:
#@title Load repo (if needed)
!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion
%cd Dreambooth-Stable-Diffusion

In [ ]:
#@title BUILD ENV
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install pytorch-lightning==1.7.6
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
!pip install -r requirements.txt

import os
os._exit(00)

In [ ]:
#@title # Required - Navigate back to the directory.
%cd Dreambooth-Stable-Diffusion

In [ ]:
#@markdown Download the 1.5 sd model with the improved vae
from IPython.display import clear_output

from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="panopstor/EveryDream",
 filename="sd_v1-5_vae.ckpt"
)

# Move the sd_v1-5_vae.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")


In [2]:
#@title # Download Regularization Images
#@markdown We’ve created the following image sets
#@markdown - `man_euler` - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
#@markdown - `man_unsplash` - pictures from various photographers
#@markdown - `person_ddim`
#@markdown - `woman_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0 <br />
#@markdown - `artstyle` - provided by Hackmans - ddim @ 50 steps, CFG 10.0 <br />

dataset="person_ddim" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "artstyle"]
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p regularization_images/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset}

In [ ]:
#@title # Training Images
#@markdown ## Upload your training images
#@markdown WARNING: Be sure to upload an even amount of images, otherwise the training inexplicably stops at 1500 steps. <br />
#@markdown - 2-3 full body
#@markdown - 3-5 upper body
#@markdown - 5-12 close-up on face  <br /> <br />
#@markdown The images should be as close as possible to the kind of images you’re trying to make (most of the time, that means no selfies).
from google.colab import files
from IPython.display import clear_output

# Create the directory
!rm -rf training_images
!mkdir -p training_images

# Upload the files
uploaded = files.upload()
for filename in uploaded.keys():
 updated_file_name = filename.replace(" ", "_")
 !mv "{filename}" "training_images/{updated_file_name}"
 clear_output()

# Tell the user what is going on
training_images_file_paths = !find training_images/*
if len(training_images_file_paths) == 0:
 print("❌ no training images found. Please upload images to training_images")
else:
 print("✅ " + str(len(training_images_file_paths)) + " training images found")


In [ ]:
#@title # Training

#@markdown This isn't used for training, just to help you remember what your trained into the model.
project_name = "project_name" #@param {type:"string"}

# MAX STEPS
#@markdown How many steps do you want to train for?
max_training_steps = 2000 #@param {type:"integer"}

#@markdown Match class_word to the category of the regularization images you chose above.
class_word = "person" #@param ["man", "person", "woman"] {allow-input: true}

#@markdown Would you like to save a model every X steps? (Example: 250 would output a trained model at 250, 500, 750 steps, etc)
save_every_x_steps = -1 #param {type:"integer"}

#@markdown If you are training a person's face, set this to True
i_am_training_a_persons_face = False #@param {type:"boolean"}

flip_p_arg = 0.0 if i_am_training_a_persons_face else 0.5


#@markdown This is the unique token you are incorporating into the stable diffusion model.
token = "firstNameLastName" #@param {type:"string"}

reg_data_root = "/content/Dreambooth-Stable-Diffusion/regularization_images/" + dataset
!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --project_name "{project_name}" \
 --training_model "model.ckpt" \
 --regularization_images "{reg_data_root}" \
 --training_images "/content/Dreambooth-Stable-Diffusion/training_images" \
 --max_training_steps {max_training_steps} \
 --class_word "{class_word}" \
 --token "{token}" \
 --flip_p {flip_p_arg} \
 --save_every_x_steps {save_every_x_steps}

In [ ]:
#@title # Copy and name the checkpoint file(s)
import re

training_images = !find training_images/*
date_string = !date +"%Y-%m-%dT%H-%M-%S"

if save_every_x_steps <= 0:
 # Copy the checkpoint into our `trained_models` folder
 directory_paths = !ls -d logs/*
 last_checkpoint_file = directory_paths[-1] + "/ckpts/last.ckpt"
 file_name = date_string[-1] + "_" + \
             project_name + "_" + \
             str(len(training_images)) + "_training_images_" + \
             str(max_training_steps) + "_max_training_steps_" + \
             token + "_token_" + \
             class_word + "_class_word.ckpt"

 file_name = file_name.replace(" ", "_")

 !mkdir -p trained_models
 !mv "{last_checkpoint_file}" "trained_models/{file_name}"

 print("Download your trained model from trained_models/" + file_name + " and use in your favorite Stable Diffusion repo!")
else:
 directory_paths = !ls -d logs/*
 checkpoints_directory = directory_paths[-1] + "/ckpts/trainstep_ckpts"
 file_paths = !ls -d "{checkpoints_directory}"/*

 for i, original_file_name in enumerate(file_paths):
  # Remove the "epoch=000000-step=0000" text
  steps = re.sub(checkpoints_directory + "/epoch=\d{6}-step=0*", "", original_file_name)

  # Remove the .ckpt
  steps = steps.replace(".ckpt", "")

  # Setup the filename
  file_name = date_string[-1] + "_" + \
                  project_name + "_" + \
                  str(len(training_images)) + "_training_images_" + \
                  steps + "_training_steps_" + \
                  token + "_token_" + \
                  class_word + "_class_word.ckpt"

  file_name = file_name.replace(" ", "_")

  # Make the directory and move the files into it.
  !mkdir -p trained_models
  !mv "{original_file_name}" "trained_models/{file_name}"

 print("Download your trained models from the 'trained_models' folder and use in your favorite Stable Diffusion repo!")

In [ ]:
#@title Save model in google drive
from google.colab import drive
drive.mount('/content/drive')

!cp trained_models/{file_name} /content/drive/MyDrive/{file_name}